In [ ]:
# Import some python libraries
import rasterio
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os

# Settings to display all outputs within notebook
%matplotlib inline
#%load_ext wurlitzer

## Test images - cloud-optimized geotiff on public cloud buckets

In [ ]:
image_url = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/047/027/LC08_L1TP_047027_20130421_20170310_01_T1/LC08_L1TP_047027_20130421_20170310_01_T1_B4.TIF'
#L8TIF = "s3://landsat-pds/L8/139/045/LC81390452014295LGN00/LC81390452014295LGN00_B1.TIF"
#L8TIFB2 = "s3://landsat-pds/L8/139/045/LC81390452014295LGN00/LC81390452014295LGN00_B2.TIF"
#image_url = "https://landsat-pds.s3.amazonaws.com/L8/139/045/LC81390452014295LGN00/LC81390452014295LGN00_B1.TIF"

In [ ]:
# Specify rasterio environment
env = rasterio.Env(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  CPL_VSIL_CURL_USE_HEAD=False,
                  CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                 )

## 0) direct download

In [ ]:
localfile = os.path.basename(image_url)
os.remove(localfile)

In [ ]:
#%%bash
!time wget -q {image_url}

In [ ]:
#%%bash

# Direct download with gdal, same settings
!time CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF gdal_translate /vsicurl/{image_url} {localfile}

In [ ]:
print(f'CPL_VSIL_CURL_USE_HEAD=NO GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR CPL_VSIL_CURL_ALLOWED_EXTENSIONS=.TIF gdal_translate /vsicurl/{image_url} {localfile}')

In [ ]:
!echo /vsicurl/{image_url}

In [ ]:
ls -l {os.path.basename(image_url)}

In [ ]:
with env:
    with rasterio.open(image_url) as src:
        print(src.profile)

## 1) rasterio into local memory

In [ ]:
%%time

with env:
    with rasterio.open(image_url) as src:
        data = src.read()

## 2) single geotiff via xarray load_rasterio

In [ ]:
%%time

with env:
    da = xr.open_rasterio(image_url)
    data = da.data

## 3) incorporate dask single chunk

In [ ]:
%%time

chunks={'band': 1, 'x': None, 'y': None}
with env:
    da = xr.open_rasterio(image_url, chunks=chunks)
    data = da.data.compute()

## 4) synchronous scheduler


In [ ]:
import dask
dask.config.set(scheduler='synchronous')  # overwrite default with single-threaded scheduler

In [ ]:
%%time

chunks={'band': 1, 'x': None, 'y': None}
with env:
    da = xr.open_rasterio(image_url, chunks=chunks)
    data = da.data.compute()

## 5) Naive Dask chunking

In [ ]:
%%time

chunks={'band': 1, 'x': 2048, 'y': 2048}
with env:
    da = xr.open_rasterio(image_url, chunks=chunks)
    data = da.data.compute()

## 6) Dask longer in row dimension

In [ ]:
%%time

chunks={'band': 1, 'x': 2048, 'y': 2048}
with env:
    da = xr.open_rasterio(image_url, chunks=chunks)
    data = da.data.compute()